# Modeling

Let's try to create several models, tune them and then choose best or try to ensemble them in better model.
 - Decicion tree
 - Random forest
 - AdaBoost
 - Linear Regression
 - Ensemble them!
 - Maybe something else?

In [15]:
import pandas as pd

df_expectancy = pd.read_csv('life_expectancy_cleaned.csv')
# for ensembly
regressors = []

In [11]:
from sklearn.model_selection import train_test_split


X, y = df_expectancy.drop(columns='Life expectancy'), df_expectancy['Life expectancy']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, train_size=0.5)

### Decision Tree

uh.

In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

parameters = {
    'criterion' : ['mse', 'mae'],
    'max_depth' : range(3, 35)
}

grid = GridSearchCV(DecisionTreeRegressor(random_state=666), parameters)
grid.fit(X_train, y_train)
mean_squared_error(y_valid, grid.predict(X_valid))

9.518174513665045

In [37]:
regressors.append(grid)
print("Best parameters:", grid.best_params_)
print(regressors)

Best parameters: {'criterion': 'mse', 'max_depth': 14}
[GridSearchCV(estimator=DecisionTreeRegressor(random_state=666),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': range(3, 35)})]


### Random forest

A little better.

In [49]:
from sklearn.ensemble import RandomForestRegressor

parameters = {
    'n_estimators' : range(100, 210, 25),
    'max_depth' : range(1, 10),
}

grid = GridSearchCV(RandomForestRegressor(random_state=666, n_jobs=-1), parameters)
grid.fit(X_train, y_train)
mean_squared_error(y_valid, grid.predict(X_valid))

4.666853644098862

In [51]:
regressors.append(grid)
print("Best parameters:", grid.best_params_)
print(regressors)

Best parameters: {'max_depth': 9, 'n_estimators': 150}
[GridSearchCV(estimator=DecisionTreeRegressor(random_state=666),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': range(3, 35)}), GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=666),
             param_grid={'max_depth': range(1, 10),
                         'n_estimators': range(100, 210, 25)})]


### AdaBoost

In [52]:
from sklearn.ensemble import AdaBoostRegressor

parameters = {
    'n_estimators' : range(100, 210, 25),
    'learning_rate' : [0.01, 0.05, 0.1, 0.3, 0.5, 1],
    'loss' : ['linear', 'square', 'exponential']
}

grid = GridSearchCV(AdaBoostRegressor(random_state=666), parameters)
grid.fit(X_train, y_train)
mean_squared_error(y_valid, grid.predict(X_valid))

10.475293911113852

In [53]:
regressors.append(grid)
print("Best parameters:", grid.best_params_)
print(regressors)

Best parameters: {'learning_rate': 0.3, 'loss': 'square', 'n_estimators': 125}
[GridSearchCV(estimator=DecisionTreeRegressor(random_state=666),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': range(3, 35)}), GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=666),
             param_grid={'max_depth': range(1, 10),
                         'n_estimators': range(100, 210, 25)}), GridSearchCV(estimator=AdaBoostRegressor(random_state=666),
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': range(100, 210, 25)})]


In [ ]:
from sklearn.linear_model import LinearRegression

